<a href="https://colab.research.google.com/github/jcs-lambda/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

#1. Load and clean the data

In [40]:
#@title Get data / imports
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

--2019-11-04 21:11:41--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.2’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2019-11-04 21:11:42 (127 KB/s) - ‘house-votes-84.data.2’ saved [18171/18171]



In [41]:
!head house-votes-84.data

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


In [42]:
!tail house-votes-84.data

democrat,n,n,y,n,n,n,y,y,n,y,y,n,n,n,y,?
democrat,y,n,y,n,n,n,y,y,y,y,n,n,n,n,y,y
republican,n,n,n,y,y,y,y,y,n,y,n,y,y,y,n,y
democrat,?,?,?,n,n,n,y,y,y,y,n,n,y,n,y,y
democrat,y,n,y,n,?,n,y,y,y,y,n,y,n,?,y,y
republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,?,n


In [43]:
house_votes_columns = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

df = pd.read_csv('house-votes-84.data', header=None, names=house_votes_columns, na_values='?')
df

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,NaN,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,NaN,NaN,NaN,NaN,n,y,y,y,n,y


In [44]:
print(df.shape)
df.describe()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [45]:
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [125]:
df = df.replace({'y':1, 'n':0})
df.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,423.000000,387.000000,424.000000,424.000000,420.000000,424.000000,421.000000,420.00000,413.000000,428.000000,414.000000,404.000000,410.000000,418.000000,407.000000,331.000000
mean,0.442080,0.503876,0.596698,0.417453,0.504762,0.641509,0.567696,0.57619,0.501211,0.504673,0.362319,0.423267,0.509756,0.593301,0.427518,0.812689
std,0.497222,0.500632,0.491140,0.493721,0.500574,0.480124,0.495985,0.49475,0.500605,0.500563,0.481252,0.494690,0.500516,0.491806,0.495327,0.390752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [47]:
sorted(df['party'].unique())

['democrat', 'republican']

In [48]:
rep = df[df['party'] == 'republican']
print(rep.shape)
rep.head()

(168, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [49]:
rep.isnull().sum()

party                    0
handicapped-infants      3
water-project           20
budget                   4
physician-fee-freeze     3
el-salvador-aid          3
religious-groups         2
anti-satellite-ban       6
aid-to-contras          11
mx-missile               3
immigration              3
synfuels                 9
education               13
right-to-sue            10
crime                    7
duty-free               12
south-africa            22
dtype: int64

In [50]:
dem = df[df['party'] == 'democrat']
print(dem.shape)
dem.head()

(267, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [51]:
dem.isnull().sum()

party                    0
handicapped-infants      9
water-project           28
budget                   7
physician-fee-freeze     8
el-salvador-aid         12
religious-groups         9
anti-satellite-ban       8
aid-to-contras           4
mx-missile              19
immigration              4
synfuels                12
education               18
right-to-sue            15
crime                   10
duty-free               16
south-africa            82
dtype: int64

#2. find an issue that democrats support more than republicans with p < 0.01

at this point, df = whole dataframe, rep = republican data, dem = democrat data

i will deal with nulls by doing calculations that discard nulls, either by default or with parameters declaring it so

## just looking at stuff

In [72]:
support_mean_by_party = pd.pivot_table(df, index='party', aggfunc='mean')
support_mean_by_party

,aid-to-contras,anti-satellite-ban,budget,crime,duty-free,education,el-salvador-aid,handicapped-infants,immigration,mx-missile,physician-fee-freeze,religious-groups,right-to-sue,south-africa,synfuels,water-project
party,,,,,,,,,,,,,,,,
democrat,0.828897,0.772201,0.888462,0.350195,0.637450,0.144578,0.215686,0.604651,0.471483,0.758065,0.054054,0.476744,0.289683,0.935135,0.505882,0.502092
republican,0.152866,0.240741,0.134146,0.981366,0.089744,0.870968,0.951515,0.187879,0.557576,0.115152,0.987879,0.897590,0.860759,0.657534,0.132075,0.506757


In [124]:
support_mean_by_party.T

party,democrat,republican
aid-to-contras,0.828897,0.152866
anti-satellite-ban,0.772201,0.240741
budget,0.888462,0.134146
crime,0.350195,0.981366
duty-free,0.637450,0.089744
education,0.144578,0.870968
el-salvador-aid,0.215686,0.951515
handicapped-infants,0.604651,0.187879
immigration,0.471483,0.557576
mx-missile,0.758065,0.115152


## here's the answer

aid-to-contras - democrats support more than republicans, confidence > 99%

In [0]:
confidence_level = 0.99

In [84]:
result = stats.ttest_ind(dem['aid-to-contras'], rep['aid-to-contras'], nan_policy='omit')
if result.pvalue < 1 - confidence_level:
  if result.statistic > 0.0:
    print('democrats support more')
  else:
    print('republicans support more')

democrats support more


#3. find an issue that republicans support more than democrats with p < 0.01

## heres the answer

crime - republicans support more than democrats, confidence > 99%

In [85]:
result = stats.ttest_ind(dem['crime'], rep['crime'], nan_policy='omit')
if result.pvalue < 1 - confidence_level:
  if result.statistic > 0.0:
    print('democrats support more')
  else:
    print('republicans support more')

republicans support more


#4. find an issue where the difference between republicans and democrats has p > 0.1

## heres the answer

water-project - level of support by democrats vs republicans is not statistically significant to a confidence level > 90%

In [95]:
result = stats.ttest_ind(dem['water-project'], rep['water-project'], nan_policy='omit')
if result.pvalue < 1 - confidence_level:
  if result.statistic > 0.0:
    print('democrats support more')
  else:
    print('republicans support more')
elif result.pvalue > 0.1:
  print('difference p-value > 1.0')
else:
  print('greater than 90% and less than 99% confidence???')
  print(result)

difference p-value > 1.0


# Stretch 1

In [0]:
def dem_vs_rep(dem_issue, rep_issue, confidence):
  result = stats.ttest_ind(dem_issue, rep_issue, nan_policy='omit')
  if result.pvalue < 1 - confidence:
    if result.statistic > 0.0:
      return('democrats support more, confidence: approx {:.2f}%'.format((1-result.pvalue)*100))
    else:
      return('republicans support more, confidence: approx {:.2f}%'.format((1-result.pvalue)*100))
  else:
    return('not enough confidence: approx {:.2f}%'.format((1-result.pvalue)*100))

## which side supports it more, confidence level 99% (p-value < .01)

democrats support more
 - handicapped infants
 - budget
 - anti-satellite ban
 - aid-to-contras
 - mx-missile
 - synfuels
 - duty-free
 - south-africa
 
republicans support more
 - physician-fee-freeze
 - el-salvador-aid
 - religious-groups
 - education
 - right-to-sue
 - crime

not enough confidence to be statistically significant
 - water-project
 - immigration


In [117]:
for column in house_columns_no_party:
  if column == 'party':
    continue
  print(column, dem_vs_rep(dem[column], rep[column], .99))
  print()

handicapped-infants democrats support more, confidence: approx 100.00%

water-project not enough confidence: approx 7.08%

budget democrats support more, confidence: approx 100.00%

physician-fee-freeze republicans support more, confidence: approx 100.00%

el-salvador-aid republicans support more, confidence: approx 100.00%

religious-groups republicans support more, confidence: approx 100.00%

anti-satellite-ban democrats support more, confidence: approx 100.00%

aid-to-contras democrats support more, confidence: approx 100.00%

mx-missile democrats support more, confidence: approx 100.00%

immigration not enough confidence: approx 91.67%

synfuels democrats support more, confidence: approx 100.00%

education republicans support more, confidence: approx 100.00%

right-to-sue republicans support more, confidence: approx 100.00%

crime republicans support more, confidence: approx 100.00%

duty-free democrats support more, confidence: approx 100.00%

south-africa democrats support more, 

## which side supports it more, confidence level 90% (p-value < .1)

water-project is the only one that has a confidence level < 90% (i.e. p-value > .1)

In [116]:
for column in house_columns_no_party:
  if column == 'party':
    continue
  print(column, dem_vs_rep(dem[column], rep[column], .9))
  print()

handicapped-infants democrats support more, confidence: approx 100.00%

water-project not enough confidence: approx 7.08%

budget democrats support more, confidence: approx 100.00%

physician-fee-freeze republicans support more, confidence: approx 100.00%

el-salvador-aid republicans support more, confidence: approx 100.00%

religious-groups republicans support more, confidence: approx 100.00%

anti-satellite-ban democrats support more, confidence: approx 100.00%

aid-to-contras democrats support more, confidence: approx 100.00%

mx-missile democrats support more, confidence: approx 100.00%

immigration republicans support more, confidence: approx 91.67%

synfuels democrats support more, confidence: approx 100.00%

education republicans support more, confidence: approx 100.00%

right-to-sue republicans support more, confidence: approx 100.00%

crime republicans support more, confidence: approx 100.00%

duty-free democrats support more, confidence: approx 100.00%

south-africa democrats